# Summarize data using Azure Databricks

Select the scored data generated by the Azure Data Factory pipeline

In [0]:
%sql
select
  *
from
  scoredflights

OriginAirportCode,Month,DayofMonth,CRSDepHour,DayOfWeek,Carrier,DestAirportCode,DepDel15,WindSpeed,SeaLevelPressure,HourlyPrecip,OriginAirportCodeIndex,OriginAirportCodeclassVec,CarrierIndex,CarrierclassVec,DestAirportCodeIndex,DestAirportCodeclassVec,label,features,rawPrediction,probability,prediction
RSW,8,9,14,5,AA,ORD,1,16.0,30.0,0.0,49.0,"Map(vectorType -> sparse, length -> 64, indices -> List(49), values -> List(1.0))",4.0,"Map(vectorType -> sparse, length -> 16, indices -> List(4), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 70, indices -> List(1), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 157, indices -> List(49, 68, 81, 150, 151, 152, 153, 154, 155), values -> List(1.0, 1.0, 1.0, 8.0, 9.0, 14.0, 5.0, 16.0, 30.0))","Map(vectorType -> dense, length -> 2, values -> List(245970.0, 314974.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4384929689951225, 0.5615070310048775))",1.0
IND,6,11,17,2,FL,MCO,0,11.0,29.0,0.0,43.0,"Map(vectorType -> sparse, length -> 64, indices -> List(43), values -> List(1.0))",9.0,"Map(vectorType -> sparse, length -> 16, indices -> List(9), values -> List(1.0))",15.0,"Map(vectorType -> sparse, length -> 70, indices -> List(15), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 157, indices -> List(43, 73, 95, 150, 151, 152, 153, 154, 155), values -> List(1.0, 1.0, 1.0, 6.0, 11.0, 17.0, 2.0, 11.0, 29.0))","Map(vectorType -> dense, length -> 2, values -> List(245970.0, 314974.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4384929689951225, 0.5615070310048775))",1.0
MCO,5,4,8,6,DL,CVG,0,0.0,29.0,0.0,10.0,"Map(vectorType -> sparse, length -> 64, indices -> List(10), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 16, indices -> List(1), values -> List(1.0))",38.0,"Map(vectorType -> sparse, length -> 70, indices -> List(38), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 157, indices -> List(10, 65, 118, 150, 151, 152, 153, 155), values -> List(1.0, 1.0, 1.0, 5.0, 4.0, 8.0, 6.0, 29.0))","Map(vectorType -> dense, length -> 2, values -> List(230561.0, 99083.0))","Map(vectorType -> dense, length -> 2, values -> List(0.6994242273482909, 0.3005757726517091))",0.0
AUS,7,21,13,7,OO,SFO,1,10.0,29.0,0.0,29.0,"Map(vectorType -> sparse, length -> 64, indices -> List(29), values -> List(1.0))",6.0,"Map(vectorType -> sparse, length -> 16, indices -> List(6), values -> List(1.0))",5.0,"Map(vectorType -> sparse, length -> 70, indices -> List(5), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 157, indices -> List(29, 70, 85, 150, 151, 152, 153, 154, 155), values -> List(1.0, 1.0, 1.0, 7.0, 21.0, 13.0, 7.0, 10.0, 29.0))","Map(vectorType -> dense, length -> 2, values -> List(245970.0, 314974.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4384929689951225, 0.5615070310048775))",1.0
SEA,4,19,21,5,WN,SMF,1,9.0,30.0,0.0,15.0,"Map(vectorType -> sparse, length -> 64, indices -> List(15), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 16, indices -> List(0), values -> List(1.0))",39.0,"Map(vectorType -> sparse, length -> 70, indices -> List(39), values -> List(1.0))",1.0,"Map(vectorType -> sparse, length -> 157, indices -> List(15, 64, 119, 150, 151, 152, 153, 154, 155), values -> List(1.0, 1.0, 1.0, 4.0, 19.0, 21.0, 5.0, 9.0, 30.0))","Map(vectorType -> dense, length -> 2, values -> List(245970.0, 314974.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4384929689951225, 0.5615070310048775))",1.0
SEA,6,13,10,4,AS,ANC,0,7.0,30.0,0.0,15.0,"Map(vectorType -> sparse, length -> 64, indices -> List(15), values -> List(1.0))",11.0,"Map(vectorType -> sparse, length -> 16, indices -> List(11), values -> List(1.0))",69.0,"Map(vectorType -> sparse, length -> 70, indices -> List(69), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 157, indices -> List(15, 75, 149, 150, 151, 152, 153, 154, 155), values -> List(1.0, 1.0, 1.0, 6.0, 13.0, 10.0, 4.0, 7.0, 30.0))",

Run the previous cell. You should see a table displayed with the scored data. Scroll all the way to the side. There you will find the prediction column containing the flight delay prediction provided by your machine learning model.

In the following cell, you will create a table that summarizes the flight delays data. Instead of containing one row per flight, this new summary table will contain one row per origin airport at a given hour, along with a count of the quantity of anticipated delays. We also join the **airport_code_location_lookup_clean** table you created at the beginning of the lab, so we can extract the airport coordinates.

In [0]:
%sql
SELECT
  OriginAirportCode,
  Month,
  DayofMonth,
  CRSDepHour,
  Sum(prediction) NumDelays,
  CONCAT(Latitude, ',', Longitude) OriginLatLong
FROM
  scoredflights s
  INNER JOIN airport_code_location_lookup_clean a ON s.OriginAirportCode = a.Airport
WHERE
  Month = 4
GROUP BY
  OriginAirportCode,
  OriginLatLong,
  Month,
  DayofMonth,
  CRSDepHour
Having
  Sum(prediction) > 1
ORDER BY
  NumDelays DESC

OriginAirportCode,Month,DayofMonth,CRSDepHour,NumDelays,OriginLatLong
ATL,4,28,16,240.0,"33.63666667,-84.42777778"
MSP,4,12,19,176.0,"44.88194444,-93.22166667"
ATL,4,14,16,144.0,"33.63666667,-84.42777778"
MSP,4,11,19,144.0,"44.88194444,-93.22166667"
MSP,4,21,19,144.0,"44.88194444,-93.22166667"
ORD,4,17,12,132.0,"41.97944444,-87.9075"
ATL,4,5,13,128.0,"33.63666667,-84.42777778"
MSP,4,5,17,128.0,"44.88194444,-93.22166667"
ATL,4,11,22,126.0,"33.63666667,-84.42777778"
ATL,4,28,15,124.0,"33.63666667,-84.42777778"


The final step is to save this summary calculation as a table, which we can later query using Power BI (in the next exercise).

In [0]:
%sql
SELECT  OriginAirportCode, Month, DayofMonth, CRSDepHour, Sum(prediction) NumDelays, CONCAT(Latitude, ',', Longitude) OriginLatLong FROM scoredflights s INNER JOIN airport_code_location_lookup_clean a ON s.OriginAirportCode = a.Airport WHERE Month = 4 GROUP BY OriginAirportCode, OriginLatLong, Month, DayofMonth, CRSDepHour  Having Sum(prediction) > 1 ORDER BY NumDelays DESC

OriginAirportCode,Month,DayofMonth,CRSDepHour,NumDelays,OriginLatLong
ATL,4,28,16,240.0,"33.63666667,-84.42777778"
MSP,4,12,19,176.0,"44.88194444,-93.22166667"
ATL,4,14,16,144.0,"33.63666667,-84.42777778"
MSP,4,11,19,144.0,"44.88194444,-93.22166667"
MSP,4,21,19,144.0,"44.88194444,-93.22166667"
ORD,4,17,12,132.0,"41.97944444,-87.9075"
ATL,4,5,13,128.0,"33.63666667,-84.42777778"
MSP,4,5,17,128.0,"44.88194444,-93.22166667"
ATL,4,11,22,126.0,"33.63666667,-84.42777778"
ATL,4,28,15,124.0,"33.63666667,-84.42777778"


In [0]:
summary = _sqldf

In [0]:
summary.write.mode("overwrite").save("/mnt/sparkcontainer/Gold/FlightDelaysSummary")

In [0]:
%sql
DROP TABLE IF EXISTS flight_delays_summary;

CREATE TABLE flight_delays_summary
USING DELTA LOCATION '/mnt/sparkcontainer/Gold/FlightDelaysSummary'

Execute the following to verify the table has data

In [0]:
%sql
select
  *
from
  flight_delays_summary

OriginAirportCode,Month,DayofMonth,CRSDepHour,NumDelays,OriginLatLong
ATL,4,28,16,240.0,"33.63666667,-84.42777778"
MSP,4,12,19,176.0,"44.88194444,-93.22166667"
ATL,4,14,16,144.0,"33.63666667,-84.42777778"
MSP,4,11,19,144.0,"44.88194444,-93.22166667"
MSP,4,21,19,144.0,"44.88194444,-93.22166667"
ORD,4,17,12,132.0,"41.97944444,-87.9075"
ATL,4,5,13,128.0,"33.63666667,-84.42777778"
MSP,4,5,17,128.0,"44.88194444,-93.22166667"
ATL,4,11,22,126.0,"33.63666667,-84.42777778"
ATL,4,28,15,124.0,"33.63666667,-84.42777778"


## Next steps

You are done executing notebooks in this lab. Please continue to the next exercise, **Exercise 7: Visualizing in Power BI Desktop** in the hands-on lab document.